# Gemini API: Function Calling
## Setup

In [1]:
!pip install -q -U google-generativeai

In [2]:
import google.generativeai as genai

In [7]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Single Function
Gemini has the ability to call user-defined functions. Let's take a look at how exactly to do this. Firstly, let us define some functions relating to a hypothetical Italian restaurant located in Berkeley.

In [3]:
def get_full_menu(service: str):
    """List all items on the menu of Gemini's Trattoria for the given service.

    Args:
        name: The type of service, lunch or dinner.
    """
    return ["Chicken Caesar Salad", "Margherita Pizza", "Spaghetti and Meatballs", "Eggplant Parmesan"]


def find_vegetarian_items(items: list[str]):
    """List all dishes in items that are vegetarian.

    Args:
        items: A list of dinner dishes.
    """
    return ["Margherita Pizza", "Eggplant Parmesan"]

def enter_restaurant():
    """You enter Gemini's Trattoria, moving the creaky door."""
    print("The door swings open, making a loud noise.")
    return True

functions = {"get_full_menu": get_full_menu,
             "find_vegetarian_items": find_vegetarian_items,
             "enter_restaurant": enter_restaurant}

After this, we go ahead and define our Gemini model. Notice how we include the argument for tools, which tells the model which functions it has available to use. We create a chat and set automatic function calling to True, which we will touch on later.

In [4]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=functions.values()
)

chat = model.start_chat(enable_automatic_function_calling=True)

Now, we go ahead and send a prompt that requires the model to call our user-defined functions.

In [8]:
response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)
print(response.text)

The dinner menu includes: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.



As we can see in the chat history, the model initially sends back a function call, to which we automatically respond, which then leads to the final model output.

In [9]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items are on Gemini's Trattoria's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'The dinner menu includes: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggp

# Behind The Scenes
In general, the processes goes as follows:

1. The user submits a query to the model.
2. The model responds with a function call.
3. The user runs the function and returns the result of the function.
4. Now, the model will either go back to Step 2 or output a final response, as seen above.

We can see this step-by-step in action by running the previous example with manual function calling, by setting the automatic function calling argument to False.

In [10]:
chat = model.start_chat(enable_automatic_function_calling=False)

response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)

part = response.candidates[0].content.parts[0]
part

function_call {
  name: "get_full_menu"
  args {
    fields {
      key: "service"
      value {
        string_value: "dinner"
      }
    }
  }
}

After this, we can reply to the model as specified in Step 3, using the functions dictionary we made earlier.

In [11]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
result = functions[part.function_call.name](**part.function_call.args)
s.update({"result": result})

function_response = glm.Part(
    function_response=glm.FunctionResponse(name="get_full_menu", response=s)
)

# Generate the next response
response = chat.send_message(function_response)
print(response.text)

The dinner menu includes Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.



## Multiple Function Calls
Our model can also call multiple functions in a row, either at the same time or one after the other, as we see in both cases below.

In [12]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are vegetarian?"
)
print(response.text)

The vegetarian options on Gemini's Trattoria's dinner menu are Margherita Pizza and Eggplant Parmesan.


In [13]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)


user -> [{'text': "What items on Gemini's Trattoria's dinner menu are vegetarian?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'find_vegetarian_items', 'args': {'items': ['Chicken Caesar Salad', 'Ma

In [14]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."
)
print(response.text)

The door swings open, making a loud noise.
I have entered Gemini's Trattoria.  The lunch and dinner menus are identical and include: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.



In [15]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}, {'function_call': {'name': 'get_full_menu', 'args': {'service': 'lunch'}}}, {'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'enter_restaurant', 'response': {'result': True}}}, {'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}, {'function_response': {'na